In [10]:
import numpy as np
import time
import json


def calculate_actuals_by_sets(match_data):
    """
    Oblicza wartość actual_home i actual_away dla meczu.
    """
    if int(match_data['winner']) == 1:
        actual_home = 1
    else:
        actual_home = 0

    return actual_home, 1 - actual_home


def calculate_predicted_delta_elo(match_data, K, K_extra):
    """
    Oblicza przewidywaną zmianę Elo na podstawie aktualnych wartości Elo i faktycznego wyniku.
    """
    expected_home = 1 / (1 + 10 ** ((match_data['away_elo'] - match_data['home_elo']) / 108))
    expected_away = 1 - expected_home

    actual_home, actual_away = calculate_actuals_by_sets(match_data)
    
    new_home_elo = K * (actual_home - expected_home)
    new_away_elo = K * (actual_away - expected_away)

    return K_extra * new_home_elo


def load_corrections(json_path='corrections.json'):
    """
    Wczytuje dane korekcyjne z pliku JSON.
    Oczekiwany format: { "1.3": 0.2, "1.4": 0.3, ... }
    """
    with open(json_path, 'r') as f:
        corrections = json.load(f)
    return corrections


def adjust_predicted_delta(p, corrections, extreme_delta, multiply_extreme):
    """
    Dopasowuje przewidywaną zmianę elo (p) na podstawie wczytanych z pliku JSON korekt.
    
    Zasada:
    1. Zaokrąglamy p do jednej cyfry po przecinku.
    2. Konwertujemy na string, żeby dopasować do klucza w słowniku.
    3. Jeśli jest w corrections, to znaczy że mamy dla tej wartości specjalną korektę.
    4. Jeśli klucza nie ma w corrections, stosujemy inne zasady.
    
    Zwracamy zaokrągloną do jednej cyfry po przecinku wartość.
    """

    p_rounded = round(p, 1)
    p_str = f"{p_rounded}"

    if p_str in corrections:
        adj = corrections[p_str]
    elif abs(p) >= extreme_delta: 
        # Zabezpieczenie na rzadkie przypadki
        adj = p_rounded * multiply_extreme
    else:
        # Brak dopasowania w słowniku korekt
        adj = p_rounded

    adj = round(adj, 1)
    
    return adj


def count_sets(match_data):
    """
    Liczy liczbę setów w meczu tenisa stołowego na podstawie danych wejściowych.

    Parameters:
    match_data (dict): Słownik zawierający dane meczu, w tym pole 'result'.

    Returns:
    int: Liczba rozegranych setów.
    """
    result = match_data.get('result', '')
    if not result:
        return "Brak wyników setów"  # Brak wyników, liczba setów to 0

    # Rozdzielenie wyników poszczególnych setów na podstawie spacji
    sets = result.strip().split(' ')

    # Filtracja ewentualnych pustych elementów (na wypadek dodatkowych spacji)
    sets = [s for s in sets if s]

    return len(sets)


def choose_parameters(n):
    if n == 3:
        K = 15
        K_extra = 1.067
        extreme_delta = 15.2
        multiply_extreme = 1.2
        correction_file_name = 'corrections_3_0.json'

        return K, K_extra, extreme_delta, multiply_extreme, correction_file_name
    elif n == 5:
        K = 10
        K_extra = 0.8017
        extreme_delta = 7.7
        multiply_extreme = 1.04
        correction_file_name = 'corrections_3_2.json'

        return K, K_extra, extreme_delta, multiply_extreme, correction_file_name
    elif n == 4:
        K = 10
        K_extra = 1.197
        extreme_delta = 11.3
        multiply_extreme = 1.06
        correction_file_name = 'corrections_3_1.json'

        return K, K_extra, extreme_delta, multiply_extreme, correction_file_name


def adjust_delta_elo_based_on_category(match_data, adjusted_delta):
    """
    Dostosowuje delta elo na podstawie kategorii meczu.

    Parameters:
    match_data (dict): Słownik zawierający dane meczu, w tym 'match_category' i 'winner'.
    adjusted_delta (float): Obliczona wcześniej zmiana elo.

    Returns:
    tuple: (delta_home, delta_away)
    """
    home_delta = adjusted_delta
    away_delta = -adjusted_delta

    match_category = match_data.get('match_category', 'group')
    winner = match_data.get('winner', 0)  # 1 dla home, 0 dla away

    if match_category == '3rd':
        if winner == 1:
            home_delta += 0.4
        else:
            away_delta += 0.4
    elif match_category == 'final':
        if winner == 1:
            home_delta += 0.8
            away_delta -= 0.6
        else:
            away_delta += 0.8
            home_delta -= 0.6
    # Jeśli 'group', nie robimy nic

    return home_delta, away_delta


def main(match_data):
    if len(match_data) > 0:
        n_sets = count_sets(match_data)
        K, K_extra, extreme_delta, multiply_extreme, correction_file_name = choose_parameters(n_sets)
        corrections = load_corrections(correction_file_name)
        pred = calculate_predicted_delta_elo(match_data, K, K_extra)
        adjusted_delta = adjust_predicted_delta(pred, corrections, extreme_delta, multiply_extreme)
        
        # Dostosowanie delta elo na podstawie kategorii meczu
        delta_home, delta_away = adjust_delta_elo_based_on_category(match_data, adjusted_delta)
        
        # Zaokrąglenie do dwóch miejsc po przecinku dla przejrzystości
        delta_home = round(delta_home, 1)
        delta_away = round(delta_away, 1)
        
        print(f"Delta Elo Home: {delta_home}")
        print(f"Delta Elo Away: {delta_away}")
        
        return delta_home, delta_away
    else:
        print("Brak danych do obliczeń.")
        return "Brak danych do obliczeń."


# Przykładowe wejście
match_data = {
    'home_elo': 764,
    'delta_elo': -5.2,
    'ft_result': '2 : 3',
    'result': '11-13 8-11 11-8 11-8 9-11',
    'away_elo': 735,
    'winner': 0,  # 0 oznacza, że wygrał away
    'match_category': 'final'
}

if __name__ == "__main__":
    main(match_data)


Delta Elo Home: -5.8
Delta Elo Away: 6.0


Dla 3:0 mamy sr. blad bezw.: **0.0528** ELO   
Dla 3:1 mamy sr. blad bezw.: **0.0452** ELO  
Dla 3:2 mamy sr. blad bezw.: **0.0314** ELO
